In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [33]:
import os
import logging
import json
import numpy as np
import faiss
import re
import pdfplumber
import torch
import whisper
import wave
import tempfile
import speech_recognition as sr
from sentence_transformers import SentenceTransformer
from typing import List, Dict, Any, Optional, Tuple
from nltk.tokenize import sent_tokenize
import nltk
import traceback
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')

def initialize_nltk():
    """Initialize required NLTK resources"""
    resources = ['punkt']
    for resource in resources:
        try:
            nltk.data.find(f'tokenizers/{resource}')
        except LookupError:
            try:
                nltk.download(resource, quiet=True)
            except Exception as e:
                raise RuntimeError(f"Failed to download NLTK resource '{resource}': {str(e)}")

# Call initialization at module level
initialize_nltk()

class ChatbotEvaluator:
    def __init__(self, chatbot):
        self.chatbot = chatbot
        self.metrics = {
            'total_queries': 0,
            'successful_responses': 0,
            'avg_response_length': 0,
            'confidence_scores': [],
            'query_types': {}
        }
    
    # Rest of the ChatbotEvaluator class remains the same...
    def evaluate_response(self, query: str, response: str) -> dict:
        """Evaluate chatbot response and update metrics"""
        evaluation = {
            'query_length': len(query.split()),
            'response_length': len(response.split()),
            'query_type': self._classify_query(query),
            'confidence_score': self._calculate_confidence(query, response)
        }
        
        self.metrics['total_queries'] += 1
        self.metrics['successful_responses'] += 1 if len(response) > 10 else 0
        self.metrics['confidence_scores'].append(evaluation['confidence_score'])
        
        query_type = evaluation['query_type']
        self.metrics['query_types'][query_type] = \
            self.metrics['query_types'].get(query_type, 0) + 1
        
        return evaluation
    
    def _classify_query(self, query: str) -> str:
        """Classify the type of query"""
        query = query.lower()
        if '?' in query:
            return 'question'
        elif any(word in query for word in ['explain', 'describe', 'define', 'elaborate', 'clarify']):
            return 'explanation'
        elif any(word in query for word in ['how', 'why', 'what', 'when', 'where', 'who']):
            return 'inquiry'
        elif any(word in query for word in ['compare', 'contrast', 'difference', 'similar']):
            return 'comparison'
        elif any(word in query for word in ['example', 'instance', 'scenario']):
            return 'example_request'
        return 'general'
    
    def _calculate_confidence(self, query: str, response: str) -> float:
        """Calculate confidence score for response"""
        query_terms = set(query.lower().split())
        response_terms = set(response.lower().split())
        term_overlap = len(query_terms & response_terms)
        semantic_relevance = self.chatbot.embedding_model.encode([query, response])
        cosine_similarity = np.dot(semantic_relevance[0], semantic_relevance[1]) / \
                          (np.linalg.norm(semantic_relevance[0]) * np.linalg.norm(semantic_relevance[1]))
        
        confidence = 0.4 * (term_overlap / len(query_terms) if query_terms else 0) + \
                    0.6 * cosine_similarity
        return float(min(confidence, 1.0))
    
    def get_performance_report(self) -> Dict[str, Any]:
        """Generate performance report"""
        return {
            'total_queries': self.metrics['total_queries'],
            'success_rate': self.metrics['successful_responses'] / max(1, self.metrics['total_queries']),
            'avg_confidence': np.mean(self.metrics['confidence_scores']) if self.metrics['confidence_scores'] else 0,
            'query_type_distribution': self.metrics['query_types']
        }



            
class DisciplinaryRuleBookChatbot:
    def __init__(
        self, 
        pdf_path: str, 
        config_path: Optional[str] = None,
        debug: bool = False,
        enable_voice: bool = True,
        whisper_model: str = "base"
    ):
        # Initialize logger
        self.logger = logging.getLogger(__name__)
        if debug:
            logging.basicConfig(level=logging.DEBUG)
        else:
            logging.basicConfig(level=logging.INFO)
            
        try:
            # Store initialization parameters
            self.pdf_path = pdf_path
            self.debug = debug
            self.enable_voice = enable_voice
            
            # Load configuration
            default_config = {
                'embedding_model': 'all-mpnet-base-v2',
                'confidence_threshold': 0.3,
                'top_k_results': 3,  # Increased for more comprehensive responses
                'min_section_length': 20,
                'max_response_sentences': 10,  # Increased for longer responses
                'min_response_sentences': 5,   # Minimum sentences in response
                'context_window': 5,          # Increased context window
                'table_extraction': True,
                'response_instructions': [
                    "Provide comprehensive explanations with examples when possible",
                    "Include relevant context and background information",
                    "Elaborate on key points with supporting details",
                    "Connect related concepts and policies",
                    "Ensure response addresses all aspects of the query",
                    "Include relevant policy references",
                    "Maintain professional tone",
                    "Use concrete examples where appropriate"
                ],
                'response_templates': {
                    'policy': "According to section {section}: {content}",
                    'clarification': "To elaborate on this topic: {content}",
                    'example': "To illustrate this point: {content}",
                    'reference': "For more detailed information, please refer to {section}."
                },
                'section_patterns': [
                    r'\n\d+[\s\.]+[A-Za-z][^\n]+\n',
                    r'\n\d+\.\d+[\s\.]+[A-Za-z][^\n]+\n',
                    r'\n\d+\.\d+\.\d+[\s\.]+[A-Za-z][^\n]+\n',
                    r'\n[A-Z][^\n]+\n',
                    r'\n[IVX]+\.?\s+[A-Z][^\n]+\n'
                ],
                'transition_phrases': [
                    "Furthermore, ",
                    "Additionally, ",
                    "Moreover, ",
                    "In relation to this, ",
                    "It's important to note that ",
                    "Building on this point, ",
                    "In this context, ",
                    "To provide more detail, ",
                    "Specifically, ",
                    "On a related note, "
                ]
            }
            
            # Load custom config if provided
            if config_path and os.path.exists(config_path):
                with open(config_path, 'r') as f:
                    custom_config = json.load(f)
                    default_config.update(custom_config)
            
            self.config = default_config
            
            # Initialize embedding model
            self.logger.info(f"Loading embedding model: {self.config['embedding_model']}")
            self.embedding_model = SentenceTransformer(self.config['embedding_model'])
            
            # Initialize voice transcription if enabled
            self.whisper = None
            if enable_voice:
                self.logger.info(f"Initializing Whisper with model: {whisper_model}")
                self.whisper = WhisperTranscriber(model_size=whisper_model, debug=debug)
            
            # Extract and process PDF content
            self.logger.info("Extracting text from PDF")
            raw_text = self._extract_raw_pdf_text()
            
            # Split into sections
            self.logger.info("Splitting text into sections")
            self.policy_sections = self._split_into_sections(raw_text)
            
            if not self.policy_sections:
                raise ValueError("No valid sections found in the PDF")
            
            # Generate embeddings
            self.logger.info("Generating section embeddings")
            self.section_embeddings = self._generate_embeddings()
            
            # Create FAISS index
            self.logger.info("Creating FAISS index")
            self.index = self._create_faiss_index()
            
            # Initialize evaluator
            self.evaluator = ChatbotEvaluator(self)
            
            self.logger.info("Initialization complete")
            
        except Exception as e:
            self.logger.error(f"Initialization failed: {str(e)}")
            raise

    def _extract_raw_pdf_text(self) -> str:
        """Extract text from PDF document with enhanced cleaning and formatting"""
        try:
            text = ""
            with pdfplumber.open(self.pdf_path) as pdf:
                for page in pdf.pages:
                    # Extract main text
                    page_text = page.extract_text() or ""
                    
                    # Clean up dots from table of contents and page numbers
                    page_text = re.sub(r'\.{2,}.*?(\d+)$', '', page_text, flags=re.MULTILINE)
                    
                    # Remove standalone page numbers
                    page_text = re.sub(r'^\s*\d+\s*$', '', page_text, flags=re.MULTILINE)
                    
                    # Clean up extra whitespace while preserving paragraph breaks
                    page_text = re.sub(r'\s+', ' ', page_text)
                    page_text = re.sub(r'\s*\n\s*', '\n', page_text)
                    
                    # Extract tables if enabled
                    if self.config['table_extraction']:
                        tables = page.extract_tables()
                        for table in tables:
                            if table and any(any(cell for cell in row) for row in table):
                                table_text = self._format_table(table)
                                if table_text:
                                    page_text += f"\n{table_text}\n"
                    
                    text += page_text + "\n"
            
            # Final cleaning
            text = self._clean_extracted_text(text)
            return text.strip()
            
        except Exception as e:
            self.logger.error(f"PDF text extraction failed: {str(e)}")
            raise
    def _clean_extracted_text(self, text: str) -> str:
        """Clean and format extracted text"""
        # Remove table of contents formatting
        text = re.sub(r'(\d+\..*?)\.{2,}\s*\d+', r'\1', text)
        
        # Fix section headers
        text = re.sub(r'(\d+\.\d*)\s*([A-Z])', r'\n\1 \2', text)
        
        # Remove duplicate section headers
        lines = text.split('\n')
        unique_lines = []
        seen_headers = set()
        
        for line in lines:
            # Check if line is a section header
            header_match = re.match(r'^\s*(\d+\.\d*\s+[A-Z].*?)$', line)
            if header_match:
                header = header_match.group(1)
                if header not in seen_headers:
                    seen_headers.add(header)
                    unique_lines.append(line)
            else:
                unique_lines.append(line)
        
        text = '\n'.join(unique_lines)
        
        # Remove empty lines and normalize spacing
        text = '\n'.join(line.strip() for line in text.split('\n') if line.strip())
        
        return text

    def _format_table(self, table: List[List[str]]) -> str:
        """Format extracted table as readable text"""
        if not table or not any(table):
            return ""
        
        formatted_rows = []
        # Clean and format header
        if table[0]:
            headers = [str(cell).strip() if cell else "" for cell in table[0]]
            formatted_rows.append(" | ".join(headers))
            formatted_rows.append("-" * len(formatted_rows[0]))  # Add separator
        
        # Format data rows
        for row in table[1:]:
            cleaned_row = [str(cell).strip() if cell else "" for cell in row]
            if any(cleaned_row):  # Only add non-empty rows
                formatted_rows.append(" | ".join(cleaned_row))
        
        return "\n".join(formatted_rows)

    def _split_into_sections(self, text: str) -> List[Dict[str, str]]:
        """Split text into sections with improved structure"""
        sections = []
        current_section = {'title': '', 'content': []}
        lines = text.split('\n')
        
        for line in lines:
            # Check if line is a section header
            header_match = re.match(r'^\s*(\d+\.?\d*)\s+([A-Z].*?)$', line)
            
            if header_match:
                # Save previous section if it exists
                if current_section['content']:
                    sections.append({
                        'title': current_section['title'],
                        'content': '\n'.join(current_section['content'])
                    })
                
                # Start new section
                current_section = {
                    'title': line.strip(),
                    'content': []
                }
            elif line.strip():
                # Add non-empty lines to current section
                current_section['content'].append(line.strip())
        
        # Add last section
        if current_section['content']:
            sections.append({
                'title': current_section['title'],
                'content': '\n'.join(current_section['content'])
            })
        
        return sections

    def _generate_embeddings(self) -> np.ndarray:
        """Generate embeddings for text sections"""
        try:
            return self.embedding_model.encode(self.policy_sections, show_progress_bar=True)
        except Exception as e:
            self.logger.error(f"Embedding generation failed: {str(e)}")
            raise

    def _create_faiss_index(self) -> faiss.IndexFlatL2:
        """Create FAISS index for efficient similarity search"""
        try:
            dimension = self.section_embeddings.shape[1]
            index = faiss.IndexFlatL2(dimension)
            index.add(self.section_embeddings)
            return index
        except Exception as e:
            self.logger.error(f"FAISS index creation failed: {str(e)}")
            raise

    def find_relevant_sections(self, query: str) -> List[Tuple[int, str]]:
        """Find relevant sections for a query with improved relevance scoring"""
        try:
            # Generate query embedding
            query_embedding = self.embedding_model.encode([query])
            
            # Search index with increased k for more comprehensive results
            k = min(self.config['top_k_results'], len(self.policy_sections))
            distances, indices = self.index.search(query_embedding, k)
            
            # Calculate confidence scores and filter results
            relevant_sections = []
            for distance, idx in zip(distances[0], indices[0]):
                confidence = 1 / (1 + distance)  # Convert distance to similarity score
                if confidence >= self.config['confidence_threshold']:
                    relevant_sections.append((idx, self.policy_sections[idx]))
            
            return relevant_sections
        
        except Exception as e:
            self.logger.error(f"Section search failed: {str(e)}")
            raise

    def generate_response(self, query: str) -> str:
        """Generate response with improved content handling"""
        try:
            relevant_sections = self.find_relevant_sections(query)
            
            if not relevant_sections:
                return ("I couldn't find any specific information about attendance rules. "
                       "Please try rephrasing your question or specify which type of attendance "
                       "rules you're interested in (e.g., class attendance, employee attendance, etc.).")
            
            # Build response with proper structure
            response_parts = []
            
            # Add introduction
            query_type = self.evaluator._classify_query(query)
            response_parts.append(self._get_introduction(query_type))
            
            # Process each relevant section
            for idx, section in relevant_sections:
                if isinstance(section, dict):
                    title = section.get('title', '')
                    content = section.get('content', '')
                else:
                    # Handle old format sections
                    title_match = re.match(r'^([\d\.]+\s+[A-Z][^\n]+)', section)
                    title = title_match.group(1) if title_match else ""
                    content = section[len(title):] if title else section
                
                # Add section title if it exists
                if title:
                    response_parts.append(f"\n{title}")
                
                # Add cleaned content
                if content:
                    content = re.sub(r'\s+', ' ', content).strip()
                    response_parts.append(content)
            
            # Combine parts with proper spacing
            response = '\n'.join(part for part in response_parts if part.strip())
            
            # Add key points
            key_sentences = self._extract_key_points(response)
            if key_sentences:
                response += "\n\nKey points:\n" + "\n".join(f"• {point}" for point in key_sentences)
            
            # Add reference footer
            if relevant_sections:
                section_refs = [f"Section {i+1}" for i, _ in relevant_sections[:3]]
                response += f"\n\nFor more detailed information, please refer to: {', '.join(section_refs)}."
            
            return response
            
        except Exception as e:
            self.logger.error(f"Response generation failed: {str(e)}")
            raise

    def _get_introduction(self, query_type: str) -> str:
        """Get appropriate introduction based on query type"""
        introductions = {
            'explanation': "Here's a detailed explanation of the attendance rules and policies. ",
            'example_request': "Let me provide specific examples of the attendance requirements. ",
            'comparison': "Let me outline the key attendance policies and requirements. ",
            'inquiry': "I'll address your question about attendance policies in detail. ",
            'general': "Here are the important attendance rules and regulations. "
        }
        return introductions.get(query_type, introductions['general'])
    def _extract_key_points(self, text: str) -> List[str]:
        """Extract key points from the response text"""
        sentences = sent_tokenize(text)
        key_points = []
        
        # Look for important sentences containing key information
        for sentence in sentences:
            # Prioritize sentences with important keywords
            if any(keyword in sentence.lower() for keyword in ['must', 'required', 'mandatory', 'important', 'essential']):
                key_points.append(sentence)
                
            # Add sentences that appear to be rules or requirements
            if re.search(r'^\d+\.|\bshall\b|\bmust\b|\bis required to\b', sentence):
                key_points.append(sentence)
        
        # Limit to 3-5 key points and remove duplicates
        key_points = list(dict.fromkeys(key_points))[:5]
        return key_points

    def _apply_response_instructions(self, response: str, query_type: str, sections: List[Tuple[int, str]]) -> str:
        """Apply enhanced response instructions for more elaborate answers"""
        try:
            formatted_response = response
            
            # Add contextual introduction based on query type
            introductions = {
                'explanation': "Let me provide a detailed explanation of this topic. ",
                'example_request': "Here's a comprehensive example with detailed context. ",
                'comparison': "Let me break down the key differences and similarities. ",
                'inquiry': "I'll address your question with comprehensive information. ",
                'general': "Let me provide detailed information about this topic. "
            }
            
            formatted_response = introductions.get(query_type, introductions['general']) + formatted_response
            
            # Enhance formatting and structure
            paragraphs = formatted_response.split('\n\n')
            enhanced_paragraphs = []
            
            for i, paragraph in enumerate(paragraphs):
                if i == 0:  # First paragraph
                    enhanced_paragraphs.append(paragraph)
                else:  # Add transitions between paragraphs
                    transition = self._get_transition_phrase(i, len(paragraphs))
                    enhanced_paragraphs.append(f"{transition} {paragraph}")
            
            formatted_response = '\n\n'.join(enhanced_paragraphs)
            
            return formatted_response
            
        except Exception as e:
            self.logger.error(f"Response formatting failed: {str(e)}")
            return response

    def _get_transition_phrase(self, position: int, total_paragraphs: int) -> str:
        """Get appropriate transition phrase based on paragraph position"""
        if position == 1:
            return "To expand on this,"
        elif position == total_paragraphs - 1:
            return "Finally,"
        else:
            transitions = self.config['transition_phrases']
            return transitions[position % len(transitions)]
            
    def process_voice_query(self, duration: int = 10) -> str:
        """Record and transcribe voice query"""
        if not self.whisper:
            raise ValueError("Voice input is not enabled")
        
        try:
            self.logger.debug("Processing voice query")
            audio_path = self.whisper.record_audio(duration)
            query = self.whisper.transcribe(audio_path)
            
            if not query:
                raise ValueError("No speech detected")
            
            return query
            
        except Exception as e:
            self.logger.error(f"Voice query processing failed: {str(e)}")
            raise

class WhisperTranscriber:
    """Handle audio recording and transcription using Whisper"""
    def __init__(self, model_size: str = "base", debug: bool = False):
        self.logger = logging.getLogger(__name__)
        self.debug = debug
        self.sample_rate = 16000
        self.channels = 1
        self.recognizer = sr.Recognizer()
        
        try:
            if self.debug:
                self.logger.debug(f"Loading Whisper model: {model_size}")
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
            self.model = whisper.load_model(model_size).to(self.device)
        except Exception as e:
            self.logger.error(f"Failed to initialize Whisper: {str(e)}")
            raise

    def record_audio(self, duration: int = 10) -> str:
        """Record audio using speech_recognition"""
        try:
            temp_file = tempfile.mktemp(suffix=".wav")
            
            # Record audio using system microphone
            with sr.Microphone() as source:
                print(f"Recording for {duration} seconds...")
                self.recognizer.adjust_for_ambient_noise(source)
                audio = self.recognizer.record(source, duration=duration)
                print("Recording complete!")
            
            # Save to WAV file
            with wave.open(temp_file, 'wb') as wf:
                wf.setnchannels(self.channels)
                wf.setsampwidth(2)  # 2 bytes per sample
                wf.setframerate(self.sample_rate)
                wf.writeframes(audio.get_wav_data())
            
            return temp_file
        except Exception as e:
            self.logger.error(f"Audio recording failed: {str(e)}")
            raise

    def transcribe(self, audio_path: str) -> str:
        """Transcribe audio file using Whisper"""
        try:
            result = self.model.transcribe(audio_path)
            return result["text"].strip()
        except Exception as e:
            self.logger.error(f"Transcription failed: {str(e)}")
            raise
        finally:
            if os.path.exists(audio_path):
                os.remove(audio_path)

def main():
    try:
        # Configurable PDF path
        pdf_path = input("Enter the full path to the PDF manual: ").strip()
        
        if not os.path.exists(pdf_path):
            print(f"Error: PDF file not found at {pdf_path}")
            return
        
        # Enable debug mode with user input
        debug_mode = input("Enable debug mode? (yes/no): ").lower().strip() == 'yes'
        
        # Configure voice input
        enable_voice = input("Enable voice input? (yes/no): ").lower().strip() == 'yes'
        whisper_model = "base"
        if enable_voice:
            print("\nAvailable Whisper models: tiny, base, small, medium, large")
            whisper_model = input("Select Whisper model size (default: base): ").strip() or "base"
        
        print("\nInitializing chatbot...")
        
        # Initialize chatbot
        chatbot = DisciplinaryRuleBookChatbot(
            pdf_path=pdf_path,
            debug=debug_mode,
            enable_voice=enable_voice,
            whisper_model=whisper_model
        )
        print(f"\nChatbot initialized successfully!")
        
        # Interactive chat loop
        while True:
            try:
                input_mode = input("\nChoose input mode (text/voice/quit/report): ").strip().lower()
                
                if input_mode == 'quit':
                    break
                
                if input_mode == 'report':
                    # Generate and display performance report
                    report = chatbot.evaluator.get_performance_report()
                    print("\nPerformance Report:")
                    print(f"Total Queries: {report['total_queries']}")
                    print(f"Success Rate: {report['success_rate']:.2%}")
                    print(f"Average Confidence: {report['avg_confidence']:.2%}")
                    print("\nQuery Type Distribution:")
                    for qtype, count in report['query_type_distribution'].items():
                        print(f"  {qtype}: {count}")
                    continue
                
                if input_mode == 'voice':
                    if not chatbot.whisper:
                        print("Voice input is not available. Please install required dependencies.")
                        continue
                    try:
                        print("\nListening... (Press Ctrl+C to stop)")
                        query = chatbot.process_voice_query()
                        print(f"\nTranscribed Query: {query}")
                    except KeyboardInterrupt:
                        print("\nRecording stopped.")
                        continue
                    except Exception as e:
                        print(f"\nVoice input error: {str(e)}")
                        continue
                elif input_mode == 'text':
                    query = input("\nEnter your query: ").strip()
                else:
                    print("Invalid input mode. Please choose 'text', 'voice', 'report', or 'quit'.")
                    continue
                
                if not query:
                    print("Please provide a valid query.")
                    continue
                
                # Generate and display response with error handling
                try:
                    response = chatbot.generate_response(query)
                    print(f"\nResponse: {response}")
                except Exception as e:
                    print(f"\nError generating response: {str(e)}")
                    if debug_mode:
                        print(f"Stack trace: {traceback.format_exc()}")
                
            except KeyboardInterrupt:
                print("\nOperation cancelled by user.")
            except Exception as e:
                print(f"\nError processing input: {str(e)}")
                if debug_mode:
                    print(f"Stack trace: {traceback.format_exc()}")

    except KeyboardInterrupt:
        print("\nExiting...")
    except Exception as e:
        print(f"\nFatal error: {str(e)}")
        if debug_mode:
            print(f"Stack trace: {traceback.format_exc()}")

if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Enter the full path to the PDF manual:  /kaggle/input/iitgmanual/UGManual.pdf
Enable debug mode? (yes/no):  yes
Enable voice input? (yes/no):  yes



Available Whisper models: tiny, base, small, medium, large


Select Whisper model size (default: base):  



Initializing chatbot...


...

Invalid input mode. Please choose 'text', 'voice', 'report', or 'quit'.



Choose input mode (text/voice/quit/report):  text

Enter your query:  suspension rules


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Response: Here are the important attendance rules and regulations. 

3.11 Students on academic probation are mandatorily put on a slow track of learn- ing. These students may register for up to 12 credits only with priority given to 1PS:ThecreditsearnedintheSummerterm(seeSection3.1)isnotconsidered
Criteria | Maximum Academic Overload ------------------------------------ CPI < 5 | No overloading allowed 5 ≤ CPI ≤ 7 | Overload up to 4 credits CPI ≥ 7 | Overload up to 8 credits availablebacklogcourses. TheP,Q,RandS parametersforthestudentswillbe adjusted accordingly. 7.

3.13 Students who are found to be unable to cope with even the slow track of learn- ing may become eligible for termination from the program. If a student already onAcademicProbation/Warningsatisfieseitherorbothoftheabove-mentioned conditionsattheendofthecurrentsemester,thentheAcademicProgramofthe student will be terminated. ConsulttheGuidelinesformultipleentryandexitformoreinformationontheexitpolicies of the institute. 


Choose input mode (text/voice/quit/report):  quit
